# EDGE-TO: Flexible Fast Python Topology Optimization For Minimum Compliance
In this notebook we provide a quick demo on how to use EDGE-TO for different scenarios.

A few important notes:
- Most memory and compute efficiencies can only be made possible for structured meshes so for the most part this is what you should be doing.
- Multi-Grid solvers for large problems are only avaible for structured meshes.
- Material models do not yet support automatic differentiation so you will have to define both forward and backwards methods for your solvers.
- Currently only linear elasticity physics is provided but you can supply your own stiffness matrix functions for different cases.

NOTE: The first time you import EDGE-TO it will take few seconds to compile some functions.

In [ ]:
import warnings
warnings.filterwarnings("ignore")

from EDGETO import *
import numpy as np
import cupy as cp

import matplotlib.pyplot as plt

import pygmsh

# Material Models
First let us take a look at the material model calsses. We provide two types of material models ```SingleMaterial``` and ```PenalizedMultiMaterial```. Both models apply a penalty exponent and compute the gradients for the solver. In both cases you can also define a penaly schedule function. By default the penalty supplied will be applied during optimization. Here we will gradually ramp up penalty from 1 (no penalty) to 3 in the first 50 steps of optimization. Note that the solver will not stop until the schedule penalty and material model penalty are the same. So even if convergence criteria is met during ramp up the solver will continue (This is good because we may quickly hit convergence criteria with low penalty). Now let us start with a single material case first.

In [ ]:
def penalty_schedule(final_penalty, iteration):
    if iteration > 50:
        return final_penalty
    else:
        return np.round(5*(1-np.cos(iteration/50*np.pi))/2)/5 * (final_penalty-1) + 1
        

# lets plot the schedule
plt.figure(figsize=(10, 5))
plt.plot([penalty_schedule(3, i) for i in range(100)])
plt.xlabel('Iteration')
plt.ylabel('Penalty')
plt.title('Penalty schedule')
plt.gca().spines['top'].set_visible(False)
plt.gca().spines['right'].set_visible(False)
plt.show()

Now let's setup the material model

In [ ]:
material = SingleMaterial(void=1e-9, penalty=3, penalty_schedule=penalty_schedule, volume_fraction=0.25)

# Geometry
There are several types of mesh each with some optimization on the compute side. First is a structured mesh which is the most specialized type of mesh and allows for the best memeory and compute efficiency. Second is an unstructured mesh with the same element types (quads, tris in 2D and hexas and tets in 3D), and finally is amixed type unstructured mesh. Let us build an example of each type of mesh in 2D for this demo. Later we will do the same for 3D but to show how to set things up let's keep things 2D.

In [ ]:
# Structured Mesh (128x64)
mesh = StructuredMesh2D(nx=128, ny=64, lx=1, ly=0.5, dtype=np.float64)

# Unstructured Uniform Mesh generate on a random domain
with pygmsh.geo.Geometry(['-setnumber', 'Mesh.Algorithm', '6',
                          '-setnumber', 'Mesh.RecombinationAlgorithm', '3',
                          '-setnumber', 'Mesh.RecombineAll', '1',
                          '-setnumber', 'Mesh.SaveWithoutOrphans', '1']) as geom:
    # Pacman parameters
    body_center = [0.5, 0.5, 0.0]
    body_radius = 0.5
    mouth_angle = 45  # Half-angle for the mouth (in degrees)
    eye_center = [0.5, 0.75, 0.0]
    eye_radius = 0.05

    # Define the mouth as a wedge
    p1 = geom.add_point(
        [
            body_center[0] + body_radius * np.cos(-np.pi/4),
            body_center[1] + body_radius * np.sin(-np.pi/4),
            0.0,
        ],
        mesh_size = 0.005
    )
    p2 = geom.add_point(
        [
            body_center[0] + body_radius * np.cos(np.pi/4),
            body_center[1] + body_radius * np.sin(np.pi/4),
            0.0,
        ],
        mesh_size = 0.005
    )
    p3 = geom.add_point(
        [
            0.0,
            0.5,
            0.0,
        ],
        mesh_size = 0.005
    )
    center_point = geom.add_point(body_center, mesh_size = 0.005)
    
    mouth_line1 = geom.add_line(center_point, p2)
    mouth_line2 = geom.add_line(p1, center_point)
    arc1 = geom.add_circle_arc(p2, center_point, p3)
    arc2 = geom.add_circle_arc(p3, center_point, p1)
    
    loop = geom.add_curve_loop([mouth_line1, arc1, arc2, mouth_line2])
    
    eye_center_point = geom.add_point(eye_center,
        mesh_size = 0.005)
    eye_right_point = geom.add_point([eye_center[0] + eye_radius, eye_center[1], 0.0],
        mesh_size = 0.005)
    eye_left_point = geom.add_point([eye_center[0] - eye_radius, eye_center[1], 0.0],
        mesh_size = 0.005)
    eye_arc1 = geom.add_circle_arc(eye_right_point, eye_center_point, eye_left_point,)
    eye_arc2 = geom.add_circle_arc(eye_left_point, eye_center_point, eye_right_point)
    
    eye_loop = geom.add_curve_loop([eye_arc1, eye_arc2])
    
    surface = geom.add_plane_surface(loop, holes=[eye_loop])
    
    # Generate the mesh
    mesh_uniform = geom.generate_mesh()

    mesh_uniform = [mesh_uniform.points[:,0:2], mesh_uniform.cells[1].data.astype(int).tolist()]

mesh_uniform = GeneralMesh(np.array(mesh_uniform[0]), np.array(mesh_uniform[1]), dtype=np.float64)

print(f"Is the mesh uniform? {mesh_uniform.is_uniform}")


# Unstructured Mixed Element Mesh generate on a random domain
with pygmsh.geo.Geometry(['-setnumber', 'Mesh.Algorithm', '6',
                          '-setnumber', 'Mesh.RecombinationAlgorithm', '5',
                          '-setnumber', 'Mesh.RecombineAll', '1',
                          '-setnumber', 'Mesh.SaveWithoutOrphans', '1']) as geom:

    # Define the mouth as a wedge
    p1 = geom.add_point(
        [
            body_center[0] + body_radius * np.cos(-np.pi/4),
            body_center[1] + body_radius * np.sin(-np.pi/4),
            0.0,
        ],
        mesh_size = 0.005
    )
    p2 = geom.add_point(
        [
            body_center[0] + body_radius * np.cos(np.pi/4),
            body_center[1] + body_radius * np.sin(np.pi/4),
            0.0,
        ],
        mesh_size = 0.005
    )
    p3 = geom.add_point(
        [
            0.0,
            0.5,
            0.0,
        ],
        mesh_size = 0.005
    )
    center_point = geom.add_point(body_center, mesh_size = 0.005)
    
    mouth_line1 = geom.add_line(center_point, p2)
    mouth_line2 = geom.add_line(p1, center_point)
    arc1 = geom.add_circle_arc(p2, center_point, p3)
    arc2 = geom.add_circle_arc(p3, center_point, p1)
    
    loop = geom.add_curve_loop([mouth_line1, arc1, arc2, mouth_line2])
    
    eye_center_point = geom.add_point(eye_center,
        mesh_size = 0.005)
    eye_right_point = geom.add_point([eye_center[0] + eye_radius, eye_center[1], 0.0],
        mesh_size = 0.005)
    eye_left_point = geom.add_point([eye_center[0] - eye_radius, eye_center[1], 0.0],
        mesh_size = 0.005)
    eye_arc1 = geom.add_circle_arc(eye_right_point, eye_center_point, eye_left_point,)
    eye_arc2 = geom.add_circle_arc(eye_left_point, eye_center_point, eye_right_point)
    
    eye_loop = geom.add_curve_loop([eye_arc1, eye_arc2])
    
    surface = geom.add_plane_surface(loop, holes=[eye_loop])

    mesh_general = geom.generate_mesh()
    mesh_general = [mesh_general.points[:,0:2], mesh_general.cells[1].data.astype(int).tolist() + mesh_general.cells[2].data.astype(int).tolist()]

mesh_general = GeneralMesh(np.array(mesh_general[0]), np.array(mesh_general[1],dtype=object), dtype=np.float64)

print(f"Is the mesh uniform? {mesh_general.is_uniform}")

A quick not on physics. The mesh object takes as input a function which is meant to compute the stiffness (K), Constitutive/Property (D), Strain-Displacement/Gradient (B) matrices and the element Area/Volume. The signiture of the expected function here is ```python f(x0: np.ndarray) -> Tuple[np.ndarray, np.ndarray, np.ndarray, float]``` where the function takes element node positions as input and returns the K, D, B, A in this order. By default this will use the built in linear ealsticity stiffness we have implemented you will need to change this if you wish to use a different type of FEA physics.

# Kernels & Filters
In our solver we need to supply a kernel for FEA calculations and a Filter kernel for filter application during optimization. Let's see how this is done:

In [ ]:
# Stiffness Kernels
kernel = StructuredStiffnessKernel(mesh)
kernel_uniform = UniformStiffnessKernel(mesh_uniform)
kernel_general = GeneralStiffnessKernel(mesh_general)

# Filter Kernels
filter_kernel = StructuredFilter2D(mesh, r_min=1.5) # This r_min is a multiple of the smallest edge size in the mesh
filter_kernel_uniform = GeneralFilter(mesh_uniform, r_min=2.0 * 0.005) # In general cases r_min is mesh space, not scaled by mesh size be cassreful!
filter_kernel_general = GeneralFilter(mesh_general, r_min=2.0 * 0.005)

# Solvers
We provide a few different solvers which you can use. In this notebook we focus on CPU solvers which are available as the following:

```python
    CHOLMOD(kernel: StiffnessKernel)
    SPLU(kernel: StiffnessKernel)
    SPSOLVE(kernel: StiffnessKernel)
    CG(kernel: StiffnessKernel, maxiter=1000, tol=1e-5)
    BiCGSTAB(kernel: StiffnessKernel, maxiter=1000, tol=1e-5)
    GMRES(kernel: StiffnessKernel, maxiter=1000, tol=1e-5)
    MultiGrid(mesh: Union[StructuredMesh2D,StructuredMesh3D],
              kernel: StiffnessKernel, maxiter=1000, tol=1e-5, n_smooth=3,
              omega=0.5 , n_level = 3, cycle='W', w_level=1, coarse_solver='splu',
              matrix_free=False, low_level_tol = 1e-8, low_level_maxiter=5000)
```

The fastest by far in most cases that can be solved practically using CPU will be CHOLMOD. For larger problems MultiGrid is best but it only works with structured meshes.

In [ ]:
solver = CHOLMOD(kernel)
solver_uniform = CHOLMOD(kernel_uniform)
solver_general = CHOLMOD(kernel_general)

# Optimizer Instance
Finally all of these will be given to an optimizer instance which will perform optimization using optimality criteria method. We will also use a Plotter instance for visualization.

In [ ]:
optimizer = TopOpt(mesh, material, kernel, solver, filter_kernel, max_iter=300, fun_tol=1e-5, ch_tol=1e-4)
optimizer_uniform = TopOpt(mesh_uniform, material, kernel_uniform, solver_uniform, filter_kernel_uniform, max_iter=300, fun_tol=1e-5, ch_tol=1e-4)
optimizer_general = TopOpt(mesh_general, material, kernel_general, solver_general, filter_kernel_general, max_iter=300, fun_tol=1e-5, ch_tol=1e-4)

plotter = Plotter(optimizer)
plotter_uniform = Plotter(optimizer_uniform)
plotter_general = Plotter(optimizer_general)

plt.figure(figsize=(8, 8))
plotter.display_mesh()
plt.axis('off')
plt.show()

plt.figure(figsize=(8, 8))
plotter_uniform.display_mesh()
plt.axis('off')
plt.show()

plt.figure(figsize=(8, 8))
plotter_general.display_mesh()
plt.axis('off')
plt.show()

## Setting Forces and Boundary Conditions
You can supply both positions and node indicies for boundary condtions. For the structured mesh we will setup the cantilever problem and for the other cases we will apply fixed points at the four points.

In [ ]:
# remove any previous BCs (not needed here)
optimizer.reset_BC()
optimizer.reset_F()

# Use nodal method for structured mesh
bc_nodes = np.where(np.isclose(mesh.nodes[:,0],0))[0]
bc_values = np.array([[1,1]]) # Broadcastable, If needed you can provide one for each point
optimizer.add_BC_nodal(bc_nodes, bc_values)

force_node = np.where(np.logical_and(np.isclose(mesh.nodes[:,0],1), np.isclose(mesh.nodes[:,1],0.25)))[0]
force = np.array([[0,-1]]) # Broadcastable, If needed you can provide one for each point
optimizer.add_F_nodal(force_node, force)

plt.figure(figsize=(8, 8))
plotter.display()
plt.axis('off')
plt.show()

In [ ]:
optimizer_uniform.reset_BC()
optimizer_uniform.reset_F()

optimizer_general.reset_BC()
optimizer_general.reset_F()

# find nodes on the boundary of the packman circle
bc_nodes = np.isclose(np.linalg.norm(mesh_uniform.nodes[:,0:2] - np.array([0.5,0.5]), axis=1),0.5)
bc_nodes = np.logical_and(bc_nodes, mesh_uniform.nodes[:,0] < 0.05)
bc_nodes = np.where(bc_nodes)[0]
bc_values = np.array([[1,1]])
optimizer_uniform.add_BC_nodal(bc_nodes, bc_values)

# apply load at the mouth of the packman
upper_mouth = np.logical_and(np.isclose(np.abs(np.dot(mesh_uniform.nodes,np.array([-1,1]))),0), mesh_uniform.nodes[:,0] > 0.5)
force_node = np.where(upper_mouth)[0]
force_nodes = force_node[np.isin(force_node, bc_nodes, invert=True)]
force = np.array([[1,-1]])/force_nodes.shape[0]/4 # Broadcastable, If needed you can provide one for each point
optimizer_uniform.add_F_nodal(force_nodes, force)

lower_mouth = np.logical_and(np.isclose(np.abs(np.dot(mesh_uniform.nodes,np.array([1,1]))),1.0), mesh_uniform.nodes[:,0] > 0.5)
force_node = np.where(lower_mouth)[0]
force_nodes = force_node[np.isin(force_node, bc_nodes, invert=True)]
force = np.array([[1,1]])/force_nodes.shape[0]/4 # Broadcastable, If needed you can provide one for each point
optimizer_uniform.add_F_nodal(force_nodes, force)

plt.figure(figsize=(8, 8))
plotter_uniform.display()
plt.axis('off')
plt.show()

bc_nodes = np.isclose(np.linalg.norm(mesh_general.nodes[:,0:2] - np.array([0.5,0.5]), axis=1),0.5)
bc_nodes = np.logical_and(bc_nodes, mesh_general.nodes[:,0] < 0.05)
bc_nodes = np.where(bc_nodes)[0]
bc_values = np.array([[1,1]])
optimizer_general.add_BC_nodal(bc_nodes, bc_values)

# apply load at the mouth of the packman
upper_mouth = np.logical_and(np.isclose(np.abs(np.dot(mesh_general.nodes,np.array([-1,1]))),0), mesh_general.nodes[:,0] > 0.5)
force_node = np.where(upper_mouth)[0]
force_nodes = force_node[np.isin(force_node, bc_nodes, invert=True)]
force = np.array([[1,-1]])/force_nodes.shape[0]/4 # Broadcastable, If needed you can provide one for each point
optimizer_general.add_F_nodal(force_nodes, force)

lower_mouth = np.logical_and(np.isclose(np.abs(np.dot(mesh_general.nodes,np.array([1,1]))),1.0), mesh_general.nodes[:,0] > 0.5)
force_node = np.where(lower_mouth)[0]
force_nodes = force_node[np.isin(force_node, bc_nodes, invert=True)]
force = np.array([[1,1]])/force_nodes.shape[0]/4 # Broadcastable, If needed you can provide one for each point
optimizer_general.add_F_nodal(force_nodes, force)

plt.figure(figsize=(8, 8))
plotter_general.display()
plt.axis('off')
plt.show()

# Optimization

In [ ]:
rho, flag, history = optimizer.optimize(save_comp_history=True)

print(f"Optimization converged: {flag}")

plt.figure(figsize=(10, 5))
plt.plot(history['comp_history'],'--' , label='compliance', color="#002b72")
plt.scatter([49], [history['comp_history'][49]], color="#ff7b00", label = "Penalty Warmup Ends", s=100)
plt.xlabel('Iteration')
plt.ylabel('Compliance')

plt.gca().spines['top'].set_visible(False)
plt.gca().spines['right'].set_visible(False)
plt.legend()
plt.show()

plt.figure(figsize=(8, 8))
plotter.display_solution(rho)
plt.axis('off')
plt.show()

In [ ]:
rho_uniform, flag, history_uniform = optimizer_uniform.optimize(save_comp_history=True)

print(f"Optimization converged: {flag}")

plt.figure(figsize=(10, 5))
plt.plot(history_uniform['comp_history'],'--' , label='compliance', color="#002b72")
plt.scatter([49], [history_uniform['comp_history'][49]], color="#ff7b00", label = "Penalty Warmup Ends", s=100)
plt.xlabel('Iteration')
plt.ylabel('Compliance')

plt.gca().spines['top'].set_visible(False)
plt.gca().spines['right'].set_visible(False)
plt.legend()
plt.show()
plt.figure(figsize=(8, 8))
plotter_uniform.display_solution(rho_uniform)
plt.axis('off')
plt.show()

In [ ]:
rho_general, flag, history_general = optimizer_general.optimize(save_comp_history=True)

print(f"Optimization converged: {flag}")

plt.figure(figsize=(10, 5))
plt.plot(history_general['comp_history'],'--' , label='compliance', color="#002b72")
plt.scatter([49], [history_general['comp_history'][49]], color="#ff7b00", label = "Penalty Warmup Ends", s=100)
plt.xlabel('Iteration')
plt.ylabel('Compliance')

plt.gca().spines['top'].set_visible(False)
plt.gca().spines['right'].set_visible(False)
plt.legend()
plt.show()

plt.figure(figsize=(8, 8))
plotter_general.display_solution(rho_general)
plt.axis('off')
plt.show()

## Plot The Fields
Now let us plot the stress and strain energy fields.

In [ ]:
strain, stress, strain_energy, von_mises = optimizer.FEA_integrals(rho>0.5)
plt.figure(figsize=(10, 5))
plotter.display_field(von_mises, rho=rho, colorbar_label="Von Mises Stress", colormap='jet', edge_color=None)
plt.axis('off')
plt.title('Von Mises Stress')
plt.show()

plt.figure(figsize=(10, 5))
plotter.display_field(strain_energy, rho=rho, colorbar_label="Strain Energy", colormap='jet', edge_color=None)
plt.axis('off')
plt.title('Strain Energy')
plt.show()

In [ ]:
strain_uniform, stress_uniform, strain_energy_uniform, von_mises_uniform = optimizer_uniform.FEA_integrals(rho_uniform>0.5)
plt.figure(figsize=(10, 10))
plotter_uniform.display_field(von_mises_uniform, rho=rho_uniform, colorbar_label="Von Mises Stress", colormap='jet', edge_color=None)
plt.axis('off')
plt.title('Von Mises Stress')
plt.show()

plt.figure(figsize=(10, 10))
plotter_uniform.display_field(strain_energy_uniform, rho=rho_uniform, colorbar_label="Strain Energy", colormap='jet', edge_color=None)
plt.axis('off')
plt.title('Strain Energy')
plt.show()

In [ ]:
strain_general, stress_general, strain_energy_general, von_mises_general = optimizer_general.FEA_integrals(rho_general>0.5)
plt.figure(figsize=(10, 10))
plotter_general.display_field(von_mises_general, rho=rho_general, colorbar_label="Von Mises Stress", colormap='jet', edge_color=None)
plt.axis('off')
plt.title('Von Mises Stress')
plt.show()

plt.figure(figsize=(10, 10))
plotter_general.display_field(strain_energy_general, rho=rho_general, colorbar_label="Strain Energy", colormap='jet', edge_color=None)
plt.axis('off')
plt.title('Strain Energy')
plt.show()